In [ ]:
import os

if not os.path.isdir("/content/Separate Corpuses/FullCorpus/") or not os.path.isdir("/joey_experiments/"):
    !gsutil -m cp -r "gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/" .
    !gsutil -m cp -r "gs://mytranslator-298419-vcm/joey_experiments/" .

Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tc.model...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tc.model...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tok.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tok.txt_true.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tok.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tok.txt_true.txt...
- [8/8 files][651.5 MiB/651.5 MiB] 100% Done                                

In [ ]:
source_language = "ru"
target_language = "be"
lang_pair = source_language+target_language
tag = "new_full"
model_type = "bpe"

use_cuda = False

if model_type == "bpe":
    # Learn BPEs on the training data.
    number_of_splits = 5000
    os.environ["data_path"] = os.path.join("joeynmt", "data", source_language + target_language)
    bpe_codes_file = "bpe.codes."+str(number_of_splits)

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["lang_pair"] = lang_pair
os.environ["tag"] = tag
os.environ["model_type"] = model_type

gdrive_path = "/content/joey_experiments/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = gdrive_path
!echo $gdrive_path

/content/joey_experiments/ru-be-new_full


In [ ]:
raw_source_file = r"/content/FullCorpus/Corpus_Rus.txt"
raw_target_file = r"/content/FullCorpus/Corpus_Bel.txt"

# They should both have the same length.
! wc -l "$raw_source_file"
! wc -l "$raw_target_file"

429479 /content/FullCorpus/Corpus_Rus.txt
429479 /content/FullCorpus/Corpus_Bel.txt


In [ ]:
%%capture
! pip install sacremoses

In [ ]:
def preprocess(filepath, language, force_preprocess=False, pretrained_tc_model=None):
  tok_file = filepath+"_tok.txt"
  tc_model = pretrained_tc_model if pretrained_tc_model else filepath+"_tc.model"
  truecased_file = tok_file+"_true.txt"

  if force_preprocess:
    ! sed -i 's/"//g' "$filepath"
    ! sacremoses tokenize -l language -e 'utf-8' < "$filepath" > "$tok_file"
    if not pretrained_tc_model:
      ! sacremoses train-truecase -m "$tc_model" -j 4 < "$tok_file"
    ! sacremoses truecase -m "$tc_model" -j 4 < "$tok_file" > "$truecased_file"

  return truecased_file, tc_model

In [ ]:
# Change the pointers to our files such that we continue to work with the tokenized and truecased data.
source_file, source_tc_model = preprocess(raw_source_file, source_language)
target_file, target_tc_model = preprocess(raw_target_file, target_language)

In [ ]:
%%capture
# installing cuda
if use_cuda:
    !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
    !sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
    !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
    !sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
    !sudo apt-get update
    !sudo apt-get -y install cuda

In [ ]:
# Install JoeyNMT
%%capture
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

In [ ]:
! cd joeynmt; python3 -m unittest

................................s.......................
----------------------------------------------------------------------
Ran 56 tests in 1.159s

OK (skipped=1)


In [ ]:
!mkdir -p joeynmt/models/${src}${tgt}_transformer_new/ && cp -r "$gdrive_path/models/"* joeynmt/models/${src}${tgt}_transformer_new/
!mkdir -p joeynmt/data/${src}${tgt}/ && cp -r "$gdrive_path/data/"* joeynmt/data/${src}${tgt}/
!cp -r joeynmt/data/${src}${tgt}/{bpe.codes.5000,vocab.ru} ./

In [ ]:
# # Test our model
# ! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/config.yaml" --output_path "$gdrive_path/models/predictions"

In [ ]:
def interactive_translate(text):
  os.environ["config"] = "config.yaml" if use_cuda else "config_non_cuda.yaml"
  ! echo "$text" | sacremoses tokenize | sacremoses truecase -m "$source_tc_model" | subword-nmt apply-bpe -c "$bpe_codes_file" --vocabulary vocab.$source_language > "joeynmt/in.txt"
  ! cd joeynmt; python3 -m joeynmt translate "$gdrive_path/models/$config" < in.txt 2> /dev/null | sacremoses detruecase 2> /dev/null | sacremoses detokenize 2> /dev/null | sed "s/ '/'/"

In [ ]:
interactive_translate("Как объяснили медработники, сделано для того, чтобы защитить стены от возможных ударов каталок.")
interactive_translate("Также на первом этаже расположены различные диагностические службы.")
interactive_translate("Например кабинет компьютерной томографии.")
interactive_translate("Аппарат, с помощью которого проводят это информативное высокотехнологичное исследование, самый современный.")
interactive_translate("Рядом - изотопная лаборатория, где выполняют радионуклидную диагностику.")

Як растлумачылі медработнікі, зроблена для таго, каб абараніць сцены ад магчымых удараў каталок.
Таксама на першым паверсе размешчаны розныя дыягнастычныя службы.
Напрыклад кабінет камп'ютарнай тамаграфіі.
Апарат, з дапамогай якога праводзяць гэта інфарматыўнае высокатэхналагічнае даследаванне, самы сучасны.
Побач — ізатопная лабараторыя, дзе выконваюць радыёнуклідную дыягнастыку.


In [ ]:
# # use_cuda = true is strongly recommended for file translation!
# def file_translate(file):
#   os.environ["config"] = "config.yaml" if use_cuda else "config_non_cuda.yaml"
#   ! sacremoses tokenize < "$file" | sacremoses truecase -m "$source_tc_model" | subword-nmt apply-bpe -c "$bpe_codes_file" --vocabulary vocab.$source_language > "joeynmt/in.txt"
#   ! cd joeynmt; python3 -m joeynmt translate "$gdrive_path/models/$config" < in.txt 2> /dev/null | sacremoses detruecase 2> /dev/null | sacremoses detokenize 2> /dev/null | sed "s/ '/'/" > joey_pred.txt

In [ ]:
# folder = f'{gdrive_path}/data/data_test/TestDatasets/AntonTestData'
# in_file_name = "2016-03_ru.txt"
# in_filepath = f'{folder}/{in_file_name}'
# file_translate(in_filepath)

In [ ]:
# !cp -r joeynmt/joey_pred.txt "$folder"
# !cp -r joeynmt/in.txt "$folder"

In [ ]:
# folder = f'{gdrive_path}/data/data_test/TestDatasets/TsimafeiMiddleCorpusTest'
# in_file_name = "MiddleCorpusTestPart_Rus.txt"
# in_filepath = f'{folder}/{in_file_name}'
# file_translate(in_filepath)

In [ ]:
# !cp -r joeynmt/joey_pred.txt "$folder"
# !cp -r joeynmt/in.txt "$folder"